In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

root = "../mnist"
train_data = datasets.MNIST(
    root=root,
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
    ) 

test_data = datasets.MNIST(
    root=root,
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
    ) 

trainset = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
testset = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

In [ ]:
import numpy
import scipy.special

class NeuralNetwork(object):

    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learning_rate

        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

    def activate_function(self, inputs):
        return scipy.special.expit(inputs)

    def train(self, inputs, targets):
        inputs = numpy.array(inputs, ndmin=2).T
        targets = numpy.array(targets, ndmin=2).T

        hidden_inputs = numpy.dot(self.wih, inputs)

        hidden_outputs = self.activate_function(hidden_inputs)

        final_inputs = numpy.dot(self.who, hidden_outputs)

        final_outputs = self.activate_function(final_inputs)

        output_errors = targets - final_outputs

        hidden_errors = numpy.dot(self.who.T, output_errors)

        self.who += self.lr * numpy.dot(
            (output_errors * final_outputs * (1.0 - final_outputs)), 
            numpy.transpose(hidden_outputs)
        )
        
        self.wih += self.lr * numpy.dot(
            (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), 
            numpy.transpose(inputs)
        )

    def query(self, inputs):
        inputs = numpy.array(inputs, ndmin=2).T

        hidden_inputs = numpy.dot(self.wih, inputs)

        hidden_outputs = self.activate_function(hidden_inputs)

        final_inputs = numpy.dot(self.who, hidden_outputs)

        final_outputs = self.activate_function(final_inputs)

        outputs = {
            index: var[0] for index, var in enumerate(final_outputs)
        }
        outputs = sorted(outputs.items(), key=lambda e: e[1], reverse=True)[0][0]
        return outputs


In [ ]:
input_nodes = 28 * 28 # image size
hidden_nodes = 100
output_nodes = 10

learning_rate = 0.35

In [ ]:
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

for data in tqdm(trainset):
    inputs, results = data

    image = inputs[0][0]
    result = results[0]
    inputs = image.reshape(len(image) ** 2)
    
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(result)] = 0.99
    n.train(inputs, targets)


In [ ]:
total = len(testset)
count = 0

for data in tqdm(testset):
    inputs, results = data

    image = inputs[0][0]
    result = int(results)
    inputs = image.reshape(len(image) ** 2)
    outputs = n.query(inputs)
    if outputs == result:
        count += 1

print('\n', count, total, count / total)
